In [27]:
import spacy
nlp = spacy.load('en_core_web_sm') 

from sutime import SUTime
sutime = SUTime(mark_time_ranges=True, include_range=True)

import csv, sqlite3
import pickle
import re

In [28]:
# variables
query_file_path = "../possible-questions.txt"

create_database_flag=False
dataset_path="../dataset/covid-19/required_only/"
database_path=r"..\dataset\covid-19\mysql_database\covid19.db"

parsed_parameter_save_path='../dataset/covid-19/parsed_parameters.pickle'

additional_stopwords=['case', 'find', 'covid', 'coronavirus', 'covid-19', 'covid19', 'world']

assign_base_words={
    'recover' : ['recover','recovery','cure','heal'],
    'death' : ['death','fatality','fatal','demise','decease','die','expire'],
    'confirm': ['confirm'],
    'active' : ['active', 'live'],
    'maximum' : ['maximum', 'high', 'max', 'maximal', 'most'],
    'minimum' : ['minimum', 'low', 'least', 'min'],
    'average' : ['average', 'avg', 'normally', 'usually', 'generally'],
    'state' : ['state', 'province'],
    'country' : ['country', 'region', 'nation', 'place']
}

In [29]:
def read_queries(query_file_path):
    queries=[]
    with open(query_file_path,"r") as f:
        for line in f.readlines():
            line = line[:-1]
            if(line):
                queries.append(line)

    return queries

def print_entities(sentence):
	doc = nlp(sentence)
	print("----> Entities:")
	for ent in doc.ents: 
		print("-------->",ent.text, ent.start_char, ent.end_char, ent.label_)

def print_tokens(sentence):
	doc = nlp(sentence) 
	print("----> Tokens:")
	for token in doc: 
		print("-------->", token.text, token.pos_, token.dep_) 

In [30]:
queries = read_queries(query_file_path)

In [31]:
def add_stopwords(additional_stopwords):
    for word in additional_stopwords:
        nlp.vocab[word].is_stop=True

def get_reverse_dict(assign_base_words):
    reverse_base_word_dict={}
    for base, l in assign_base_words.items():
        for item in l:
            doc = nlp(item)
            item = doc[0].lemma_
            reverse_base_word_dict[item]=base

    return reverse_base_word_dict

In [32]:
add_stopwords(additional_stopwords)
reverse_base_word_dict = get_reverse_dict(assign_base_words)

In [33]:
def csv_to_table(cur, path, csv_name, table_name):
    with open(dataset_path+csv_name,'r') as fin:
        dr = csv.DictReader(fin)
        to_db=[tuple(i.values()) for i in dr]
    
    count=len(dr.fieldnames)
    bindings="?, "*count

    cur.executemany("INSERT INTO "+table_name+" VALUES ("+bindings[:-2]+");", to_db)

def create_database():
    con = sqlite3.connect(database_path)
    cur = con.cursor()

    tables=[("worldwide_aggregate"),("reference"),("timeseries"),("us")]
    for table in tables:
        cur.execute("DROP TABLE IF EXISTS "+table+";")

    cur.execute("create table worldwide_aggregate(Date Date NOT NULL, Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL, Increase_rate FLOAT default NULL, PRIMARY KEY (Date));")

    csv_to_table(cur,dataset_path,"worldwide-aggregate.csv", tables[0])

    cur.execute("create table us(Date Date NOT NULL, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Confirmed BIGINT NOT NULL, Deaths BIGINT NOT NULL, Country_Region VARCHAR(100) NOT NULL, PRIMARY KEY (Date, Admin2, Province_State));")

    csv_to_table(cur,dataset_path,"us_simplified.csv", tables[3])

    cur.execute("create table reference(UID INT NOT NULL, iso2 VARCHAR(20), iso3 VARCHAR(20), code3 INT, FIPS INT, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Country_Region VARCHAR(100) NOT NULL, Lat FLOAT NOT NULL, Long_ FLOAT NOT NULL, Combined_Key VARCHAR(100), Popolation BIGINT NOT NULL, PRIMARY KEY (UID));")

    csv_to_table(cur,dataset_path,"reference.csv", tables[1])

    cur.execute("create table timeseries(Date Date NOT NULL, Country_Region VARCHAR(100) NOT NULL, Province_State VARCHAR(100), Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL, PRIMARY KEY (Date, Country_Region, Province_State));")

    csv_to_table(cur,dataset_path,"time-series-19-covid-combined.csv", tables[2])

    con.commit()

    # with open('../dataset/covid-19/mysql_database/dump.sql','w') as fp:
    #     for line in con.iterdump():
    #         fp.write('%s\n' % line)

    con.close()

In [34]:
if create_database_flag:
    create_database()

In [35]:
def generate_country_state_list():
    con = sqlite3.connect(database_path)

    cur = con.execute("SELECT DISTINCT Province_State, Country_Region FROM reference;")
    # cur = con.execute("SELECT count(*) FROM reference;")

    countries_only=[]
    states_only=[]
    state_country_dict={}

    for row in cur:
        countries_only.append(row[1].lower())
        if row[0]:
            states_only.append(row[0].lower())
            state_country_dict[row[0].lower()]=row[1].lower()

    con.close()
    return countries_only, states_only, state_country_dict

In [36]:
countries_only, states_only, state_country_dict=generate_country_state_list()

In [63]:
def identify_places(places):
    place_dict={
        'no_match':[],
        'states':[],
        'countries':[]
    }

    for place in places:
        place=place.lower()
        if place in states_only:
            place_dict['states'].append(place)
        elif place in countries_only:
            place_dict['countries'].append(place)
        else:
            place_dict['no_match'].append(place)
    
    return place_dict

def get_places(entities):
    places=[]
    for ent in entities:
        if ent.label_ == 'GPE':
            places.append(ent.text)
    
    place_dict=identify_places(places)
    
    return place_dict

def find_regex(s):
    X = re.search('....-..-..', s)
    if(X):
        return X.group()
    
    X = re.search('....-..', s)
    if(X):
        return X.group()

def process_interval(time):
    # print(time)
    if(time['begin']>time['end']):
        time['begin'], time['end']=time['end'], time['begin']

    start_time=time['begin']
    YMD=start_time.split('-')
    if(len(YMD)==2):
        YMD.append('XX')

    YMD[1]=YMD[1] if YMD[1]!='XX' else '01'
    YMD[2]=YMD[2] if YMD[2]!='XX' else '01'
    # YMD.reverse()
    begin='-'.join(YMD)

    end_time=time['end']
    YMD=end_time.split('-')
    if(len(YMD)==2):
        YMD.append('XX')

    YMD[1]=YMD[1] if YMD[1]!='XX' else '12'
    YMD[2]=YMD[2] if YMD[2]!='XX' else '31'
    # YMD.reverse()
    end='-'.join(YMD)

    time['begin']=begin
    time['end']=end

    return time

def get_time_duration(query):
    time={}
    parsed=sutime.parse(query)

    if len(parsed)==1:
        item=parsed[0]

        if item['type']=='DATE':
            if item['value'] != 'PRESENT_REF':
                time['begin']=item['value']
                time['end']=item['value']

        elif item['type']=='DURATION' and type(item['value'])==type(time):
            time['begin']=item['value']['begin']
            time['end']=item['value']['end']
    
    elif len(parsed)==2 and parsed[0]['type']=='DATE' and parsed[1]['type']=='DATE':
        time['begin']=min(parsed[0]['value'], parsed[1]['value'])
        time['end']=max(parsed[0]['value'], parsed[1]['value'])

    # print(parsed)
    # for item in parsed:
    #     time.append(item['value'])

    if len(time)==0:
        time['begin']='XXXX-XX-XX'
        time['end']='XXXX-XX-XX'
    
    
    time['begin']=find_regex(time['begin'])
    time['end']=find_regex(time['end'])
    # print(time)
    time=process_interval(time)

    return time

def remove_unnecessary(query):
    doc=nlp(query)

    for ent in doc.ents:
        if ent.label_=='GPE' or ent.label_=='DATE':
            query=query.replace(ent.text,"")
    # print(query)

    query=query.lower()
    doc=nlp(query)
    query=' '.join([token.lemma_ for token in doc])

    for word,base in reverse_base_word_dict.items():
        # query=query.replace(word,base)
        query = ' '.join(base if w == word else w for w in query.split())

    doc=nlp(query)
    for token in doc:
        if token.is_stop==True or token.dep_=='prep' or token.dep_=='punct':
            # query=query.replace(token.text,"")
            query = ' '.join("" if w == token.text else w for w in query.split())
    
    return query

def get_case_and_function_type(query):
    processed_query = remove_unnecessary(query)
    # print(query)
    # print(processed_query)

    case_types = ['confirm','recover','death','increase rate','active']
    function_types = ['maximum', 'minimum', 'average', 'sum']

    final_case=""
    for case in case_types:
        if(processed_query.find(case)>=0):
            final_case = case
            break
    
    if final_case=="":
        final_case='confirm'

    final_func=""
    for func in function_types:
        if(processed_query.find(func)>=0):
            final_func = func
            break
    
    if final_func=="":
        final_func='sum'
    
    return final_case, final_func

def get_operation_type(query):
    operation_type = ['state', 'country']

    query=query.lower()
    doc=nlp(query)
    query=' '.join([token.lemma_ for token in doc])

    for word,base in reverse_base_word_dict.items():
        # query=query.replace(word,base)
        query = ' '.join(base if w == word else w for w in query.split())
    
    doc=nlp(query)
    for token in doc:
        if token.dep_!='compound':
            if token.text == 'state' or token.text == 'country':
                return token.text

    return 'cases'

def parse_parameters(query):
    doc=nlp(query)
    entities=doc.ents

    place = get_places(entities)
    time_duration = get_time_duration(query)
    case_type, function_type = get_case_and_function_type(query)
    operation_type = get_operation_type(query)

    parameters={
        'query':query,
        'Place':place,
        'Time Duration': time_duration,
        'Case Type': case_type,
        'Function Type': function_type,
        'Operation Type': operation_type
        }
    return parameters

In [64]:
def process_query(query):
    # print(query)
    parameters=parse_parameters(query)
    # print_entities(query)
    #print_tokens(sent)
    # doc=nlp(query)
    # print([chunk.text for chunk in doc.noun_chunks])
    # print(parameters)
    # print()
    return parameters

In [65]:
parameter_list=[]
for q in queries:
    parameter_list.append(process_query(q))

with open(parsed_parameter_save_path, 'wb') as f:
    pickle.dump(parameter_list, f)

In [66]:
parameter_list

[{'query': 'Which country saw highest number of death in the month of April?',
  'Place': {'no_match': [], 'states': [], 'countries': []},
  'Time Duration': {'begin': 'XXXX-04-01', 'end': 'XXXX-04-31'},
  'Case Type': 'death',
  'Function Type': 'maximum',
  'Operation Type': 'country'},
 {'query': 'Which country was last to be affected by virus?',
  'Place': {'no_match': [], 'states': [], 'countries': []},
  'Time Duration': {'begin': 'XXXX-01-01', 'end': 'XXXX-12-31'},
  'Case Type': 'confirm',
  'Function Type': 'sum',
  'Operation Type': 'country'},
 {'query': 'Which country was first to reach 1lakh cases?',
  'Place': {'no_match': [], 'states': [], 'countries': []},
  'Time Duration': {'begin': 'XXXX-01-01', 'end': 'XXXX-12-31'},
  'Case Type': 'confirm',
  'Function Type': 'sum',
  'Operation Type': 'country'},
 {'query': 'Which country was first to reach 50000 deaths?',
  'Place': {'no_match': [], 'states': [], 'countries': []},
  'Time Duration': {'begin': 'XXXX-01-01', 'end':